In [ ]:
!pip install -U "transformers>=4.34.0" "datasets>=2.15.0" "accelerate>=0.21.0" \
            "torch>=2.0.0" "pandas>=2.0.0" "numpy>=1.24.0" "scikit-learn>=1.3.0" \
            "joblib>=1.3.0" "pyarrow>=14.0.0" "fastparquet>=2023.8.0" "tqdm>=4.66.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 760.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 151.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer
import pandas as pd

def load_parquet_dataset(parquet_path, text_field="code", label_field="label", streaming=False):
    """
    Load a parquet file as either a full in-memory dataset or a streaming dataset.
    - If streaming=True: returns an IterableDataset (low memory use)
    - If streaming=False: loads entire dataset into memory
    """
    if streaming:
        print(f"🔄 Loading {parquet_path} in streaming mode...")
        dataset = load_dataset("parquet", data_files=parquet_path, streaming=True)
        return dataset["train"]
    else:
        print(f"📦 Loading {parquet_path} fully into memory...")
        dataset = load_dataset("parquet", data_files=parquet_path, split="train")
        return dataset


class CodeDatasetPreprocessor:
    """
    Tokenizes code or text fields in Hugging Face datasets for transformer models.
    """

    def __init__(self, model_name_or_path, max_length=512, text_field="text"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
        self.max_length = max_length
        self.text_field = text_field

    def tokenize_batch(self, examples):
        return self.tokenizer(
            examples[self.text_field],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )

    def prepare(self, dataset):
        """
        Apply tokenization to a Hugging Face Dataset (non-streaming).
        """
        return dataset.map(
            self.tokenize_batch,
            batched=True,
            remove_columns=[self.text_field]
        )

In [ ]:
# src/metrics.py
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import math
from typing import Dict

def classification_metrics(preds, labels) -> Dict:
    """Compute accuracy, precision, recall, and f1 (weighted).
    preds can be logits (ndarray) or 1d predicted labels."""
    import numpy as _np
    preds_arr = _np.array(preds)
    if preds_arr.ndim > 1:
        pred_labels = preds_arr.argmax(axis=1)
    else:
        pred_labels = preds_arr
    acc = accuracy_score(labels, pred_labels)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, pred_labels, average='weighted', zero_division=0)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

def perplexity(loss):
    return math.exp(loss)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig

def get_sequence_classification_model(name_or_path, num_labels, from_pretrained=True):
    if from_pretrained:
        model = AutoModelForSequenceClassification.from_pretrained(name_or_path, num_labels=num_labels)
    else:
        config = AutoConfig.from_pretrained(name_or_path, num_labels=num_labels)
        model = AutoModelForSequenceClassification.from_config(config)
    return model

In [ ]:
import torch

if torch.cuda.is_available():
    print("✅ GPU is available!")
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("❌ No GPU found — go to Runtime > Change runtime type > GPU.")


❌ No GPU found — go to Runtime > Change runtime type > GPU.


In [ ]:
!pip install -U transformers accelerate datasets

In [ ]:
# src/train_binary.py
import os
import shutil
import numpy as np
import torch
import torch_xla.core.xla_model as xm
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset, IterableDataset

# ---------- helper ----------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return classification_metrics(np.array(logits), np.array(labels))

def choose_output_dir(base_dir="/content/drive/MyDrive/experiments/binary"):
    try:
        total, used, free = shutil.disk_usage("/")
        free_gb = free / (1024 ** 3)
        if free_gb < 5:
            alt_dir = "/content/tmp_models"
            print(f"⚠️ Low disk space ({free_gb:.2f} GB). Using {alt_dir}")
            return alt_dir
        return base_dir
    except Exception as e:
        print(f"⚠️ Could not check disk space ({e}), using default output dir.")
        return base_dir

# ---------- main ----------
def main():
    model_name = "microsoft/codebert-base"
    train_path = "/content/drive/MyDrive/train.parquet"
    val_path   = "/content/drive/MyDrive/val.parquet"
    out_dir    = choose_output_dir()
    num_labels = 2
    epochs     = 3
    batch_size = 16   # TPU can handle bigger batches
    lr         = 2e-5

    # Detect TPU device
    device = xm.xla_device()
    print(f"💻 Using TPU device: {device}")

    os.makedirs(out_dir, exist_ok=True)

    # --- Load streaming datasets ---
    print("📂 Loading streaming datasets (memory-efficient)...")
    train_stream = load_parquet_dataset(train_path, streaming=True)
    val_stream   = load_parquet_dataset(val_path, streaming=True)

    # --- Preprocessor ---
    preproc = CodeDatasetPreprocessor(model_name, max_length=256, text_field="code")

    def tokenize_batch(batch):
        return preproc.tokenizer(
            batch["code"],
            truncation=True,
            padding=False,
            max_length=256,
        )

    print("🔡 Applying tokenizer (map in streaming mode)...")
    train_tok = train_stream.map(tokenize_batch, batched=True)
    val_tok   = val_stream.map(tokenize_batch, batched=True)

    # --- Initialize model ---
    print("🚀 Initializing model...")
    model = get_sequence_classification_model(model_name, num_labels=num_labels)
    model.to(device)

    # --- TrainingArguments for TPU ---
    print("⚙️ Setting up training arguments...")
    args = TrainingArguments(
        output_dir=out_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_steps=100,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        report_to="none",
        tpu_num_cores=8,     # important for TPU
        bf16=True,           # TPUs prefer bfloat16
        dataloader_drop_last=True,
    )

    data_collator = DataCollatorWithPadding(tokenizer=preproc.tokenizer)

    print(f"🏋️ Training on TPU (output: {out_dir})...")
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_tok,
        eval_dataset=val_tok,
        tokenizer=preproc.tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    print("💾 Saving model and tokenizer...")
    trainer.save_model(out_dir)
    preproc.tokenizer.save_pretrained(out_dir)
    print(f"✅ Training complete! Model saved to: {out_dir}")

# ---------- entry ----------
if __name__ == "__main__":
    main()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
